In [1]:
import org.apache.spark.sql.types.{StructType, StringType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [10]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-random_forest-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("timestamp", TimestampType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 2)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-random_forest-ml-model
dataSchema = StructType(StructField(tweet,StringType,true), StructField(timestamp,TimestampType,true))
inputDF = [tweet: string, timestamp: timestamp]


[tweet: string, timestamp: timestamp]

In [11]:
// загружаем модель в объект
val model = PipelineModel.load(modelPath)

model = pipeline_d732f58e5dab


pipeline_d732f58e5dab

In [12]:
// функция для получения элемента 1 из probability
val getProbability = udf((p: org.apache.spark.ml.linalg.Vector) => p.toArray(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [17]:
// запускаем модель для каждой микро-пачки
// выводим элемент 1 из "probability"
inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    println (batchId)
    // запускаем модель
    model.transform(batchDF)
        .select($"timestamp", getProbability($"probability").alias("probability_1"), $"prediction")
        .withColumn("Negative", when($"prediction" === 1, 1).otherwise(0))
        .withColumn("Positive", when($"prediction" === 0, 1).otherwise(0))
        .show(false)
}
.start()

0


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6e282dbd

+-----------------------+-------------------+----------+--------+--------+
|timestamp              |probability_1      |prediction|Negative|Positive|
+-----------------------+-------------------+----------+--------+--------+
|2020-02-08 00:40:49.778|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:40:49.778|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:40:49.778|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:40:49.778|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:40:49.778|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:40:49.778|0.40431175924111457|0.0       |0       |1       |
|2020-02-08 00:40:49.778|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:40:49.778|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:40:49.778|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:40:49.778|0.549917097257528  |1.0       |1       |0       |
|2020-02-08 00:40:49.778|

6
+-----------------------+-------------------+----------+--------+--------+
|timestamp              |probability_1      |prediction|Negative|Positive|
+-----------------------+-------------------+----------+--------+--------+
|2020-02-08 00:41:24.842|0.43877432910411035|0.0       |0       |1       |
|2020-02-08 00:41:24.842|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:41:24.842|0.5714848214956812 |1.0       |1       |0       |
|2020-02-08 00:41:24.842|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:41:24.842|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:41:24.842|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:41:24.842|0.40431175924111457|0.0       |0       |1       |
|2020-02-08 00:41:24.842|0.6031716035513216 |1.0       |1       |0       |
|2020-02-08 00:41:24.842|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:41:24.842|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:41:24.84

12
+-----------------------+------------------+----------+--------+--------+
|timestamp              |probability_1     |prediction|Negative|Positive|
+-----------------------+------------------+----------+--------+--------+
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:41:57.188|0.54991709

...

24
+-----------------------+-------------------+----------+--------+--------+
|timestamp              |probability_1      |prediction|Negative|Positive|
+-----------------------+-------------------+----------+--------+--------+
|2020-02-08 00:42:58.484|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:42:58.484|0.42451302529307167|0.0       |0       |1       |
|2020-02-08 00:42:58.484|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:42:58.484|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:42:58.484|0.4529542882105761 |0.0       |0       |1       |
|2020-02-08 00:42:58.484|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:42:58.484|0.40431175924111457|0.0       |0       |1       |
|2020-02-08 00:42:58.484|0.4527909250649727 |0.0       |0       |1       |
|2020-02-08 00:42:58.484|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:42:58.484|0.43258965901301566|0.0       |0       |1       |
|2020-02-08 00:42:58.4

30
+-----------------------+------------------+----------+--------+--------+
|timestamp              |probability_1     |prediction|Negative|Positive|
+-----------------------+------------------+----------+--------+--------+
|2020-02-08 00:43:29.603|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.4529542882105761|0.0       |0       |1       |
|2020-02-08 00:43:29.603|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.5092813398247178|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.5028275172582966|1.0       |1       |0       |
|2020-02-08 00:43:29.603|0.45295428

36
+-----------------------+-------------------+----------+--------+--------+
|timestamp              |probability_1      |prediction|Negative|Positive|
+-----------------------+-------------------+----------+--------+--------+
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:43:59.695|0.43953392440289424|0.0       |0       |1       |
|2020-02-08 00:43:59.6

42
+-----------------------+-------------------+----------+--------+--------+
|timestamp              |probability_1      |prediction|Negative|Positive|
+-----------------------+-------------------+----------+--------+--------+
|2020-02-08 00:44:31.459|0.42451302529307167|0.0       |0       |1       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.459|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:44:31.4

48
+----------------------+-------------------+----------+--------+--------+
|timestamp             |probability_1      |prediction|Negative|Positive|
+----------------------+-------------------+----------+--------+--------+
|2020-02-08 00:45:01.13|0.40431175924111457|0.0       |0       |1       |
|2020-02-08 00:45:01.13|0.43877432910411035|0.0       |0       |1       |
|2020-02-08 00:45:01.13|0.5311054170301976 |1.0       |1       |0       |
|2020-02-08 00:45:01.13|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:45:01.13|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:45:01.13|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:45:01.13|0.42451302529307167|0.0       |0       |1       |
|2020-02-08 00:45:01.13|0.45679723314777404|0.0       |0       |1       |
|2020-02-08 00:45:01.13|0.549917097257528  |1.0       |1       |0       |
|2020-02-08 00:45:01.13|0.5028275172582966 |1.0       |1       |0       |
|2020-02-08 00:45:01.13|0.502827517

In [13]:
// вывод кол-ва негативных и позитивных твитов за 10 сек скользящим окном
inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    println (batchId)
    model.transform(batchDF)
        .withColumn("Negative", when($"prediction" === 1, 1).otherwise(0))
        .withColumn("Positive", when($"prediction" === 0, 1).otherwise(0))
        .withWatermark("timestamp", "15 seconds")
        .groupBy(window($"timestamp", "10 seconds", "2 seconds"))
        .agg(sum($"Negative").alias("Negative"), sum($"Positive").alias("Positive"), count("prediction").alias("Total"))
        .orderBy($"window")
        .show(false)
}
.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5d894f1a

0
+------------------------------------------+--------+--------+-----+
|window                                    |Negative|Positive|Total|
+------------------------------------------+--------+--------+-----+
|[2020-02-08 00:40:40, 2020-02-08 00:40:50]|17      |3       |20   |
|[2020-02-08 00:40:42, 2020-02-08 00:40:52]|17      |3       |20   |
|[2020-02-08 00:40:44, 2020-02-08 00:40:54]|17      |3       |20   |
|[2020-02-08 00:40:46, 2020-02-08 00:40:56]|39      |4       |43   |
|[2020-02-08 00:40:48, 2020-02-08 00:40:58]|39      |4       |43   |
|[2020-02-08 00:40:50, 2020-02-08 00:41:00]|22      |1       |23   |
|[2020-02-08 00:40:52, 2020-02-08 00:41:02]|22      |1       |23   |
|[2020-02-08 00:40:54, 2020-02-08 00:41:04]|22      |1       |23   |
+------------------------------------------+--------+--------+-----+

1
+------------------------------------------+--------+--------+-----+
|window                                    |Negative|Positive|Total|
+----------------------------

In [8]:
// эксперименты со скользящим окном
inputDF
    .withWatermark("timestamp", "15 seconds")
    .groupBy(window($"timestamp", "10 seconds", "2 seconds")).count()
    .writeStream.format("console").option("truncate", false)
    .start()

lastException: Throwable = null


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@71ec699b

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----+
|window|count|
+------+-----+
+------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|window|count|
+------+-----+
+------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|window|count|
+------+-----+
+------+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|window|count|
+------+-----+
+------+-----+

-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2020-02-08 00:40:42, 2020-02-08 00:40:52]|20   |
|[2020-02-08 00:40:40, 2020-02-08 00:40:50]|20   |
+---------------